# 🔋 **Extended Kalman Filter for Generating Open Circuit Voltage vs SoC Relationship**
***

### The notebook being divided into phases 📖
🧪 **Data Proceesing** 

📊 **Data Visualization** 

📉 **Data Fitting** 

#### 📚Library Imports

In [1]:
import serial
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import curve_fit

## 🧪Phase 1 - Data Processing
>   - Recieves open circuit voltage and current through bleeder resistor data from Measurement-to-Serial.ino
>   - Extended Kalman Filter is applied to the data to process SoC
>   - Open Circuit Voltage and SoC is stored in a CSV

#### 💾 Ardunio NANO connected to COM5

In [2]:
arduino_port = "COM5"
baud = 9600 
fileName="V_oc&SoC.csv"
ser = serial.Serial(arduino_port, baud)
file = open(fileName, "a")

#### 📲 Dual Polarization Model and EKF Parameters

In [5]:
del_t = 100
SoC = 1;
V_ct = 0.007;
V_df = 0.004;
tau_ct = 4;
tau_df = 100;
R_ct = 0.001;
R_df = 0.01;
R_0 = 0.2;
R_l = 2.2;
Charge = 0.6*3600;


x = np.array([[SoC],
              [V_ct],
              [V_df]]);

A = np.array([[1, 0, 0],
              [0, np.exp(-(del_t/1000)/tau_ct), 0],
              [0, 0, np.exp(-(del_t/1000)/tau_df)]]);
              
B = np.array([[-(del_t/1000)/Charge],
              [R_ct*(1-np.exp(-(del_t/1000)/tau_ct))],
              [R_df*(1-np.exp(-(del_t/1000)/tau_df))]]);

P = np.array([[1e-25, 0, 0],
              [0, 1e-23, 0],         
              [0, 0, 1e-23]]);

R = np.array([1e-7]);

Q = np.array([[0, 0, 0],
              [0, 0, 0],         
              [0, 0, 0]]);

C = np.array([1, -1, -1]);

#### 🧮 **Extended Kalman Filter**
> - Data Estimation
> - Data Storage

[1] Algorithm based on Carlo Taborelli, Simona Onori. (2014)."[State of Charge Estimation
 Using Extended Kalman Filters for Battery Management System](https://ieeexplore.ieee.org/document/7056126)". 
 
 
[2] Parameter Update based on 
    yu, Zhihao & Huai, Ruituo & Xiao, Linjing. (2015). 
    "[State-of-Charge Estimation for Lithium-Ion Batteries 
    Using a Kalman Filter Based on Local Linearization](https://www.researchgate.net/publication/282462924_State-of-Charge_Estimation_for_Lithium-Ion_Batteries_Using_a_Kalman_Filter_Based_on_Local_Linearization)".
    

In [ ]:
SoC = 1
I = 0
x[0][0]=1
n=1

while x[0][0] > 0.01:
    getData=str(ser.readline())[2:][:-5]
    I = getData[5:][:]
    I = float(I)
    
    #Extended Kalman Estimator
    
    #19a
    x_pred = np.dot(A, x)+(B*I)
    
    #19b
    P_pred = np.dot(np.dot(A, P), np.transpose(A)) + Q
    
    #Parameter Update
    C[0] = (float(getData[0:][:-5]))/x_pred[0][0]
                                
    #20a
    L = np.dot(P_pred, np.transpose(C))/(np.dot(C, np.dot(P_pred, np.transpose(C)))+R)
    
    #20b
    x = x_pred + (L*(I*R_l - np.dot(C, x_pred) + I*R_0))
    
    #20c
    P = np.dot((np.identity(3, dtype = float) - np.dot(L, C)), P_pred)
    
    SoC = x[0][0]
    
    if n==1:
        SoC = 1
        x[0][0] = 1
        n=0
    
    #Write to CSV
    file = open(fileName, "a")
    file.write(getData[0:][:-5]+",")
    file.write(str(SoC))
    file.write("\n")
    
file.close()

## 📊 Phase 2 - Data Visualization
>   - Reads data from CSV
>   - Performs a Matplolib mapping

In [ ]:
values = pd.read_csv("V_oc&SoC.csv")
plt.plot(values["SoC"], values["Voc"], color='orange', linewidth=3)
plt.xlabel("$SoC(%)$", fontsize=13)
plt.ylabel("$V_{OC}$", fontsize=13)
plt.title("$V_{OC} vs SoC(%)$", fontsize=15)

## 📉 Phase 3 - Data Fitting
>  Fits curve to the generated plot for further SoC estimation using open circuit voltage measurement

##### Fitting equation based on 
##### $$OCV(SoC) = K_0-K_{1}/SoC-K_{2}SoC+K_{3}ln(SoC)+K_{4}ln(1-SoC)$$
pg. 7 of yu, Zhihao & Huai, Ruituo & Xiao, Linjing. (2015). 
    "[State-of-Charge Estimation for Lithium-Ion Batteries 
    Using a Kalman Filter Based on Local Linearization](https://www.researchgate.net/publication/282462924_State-of-Charge_Estimation_for_Lithium-Ion_Batteries_Using_a_Kalman_Filter_Based_on_Local_Linearization)".



In [ ]:
def guess_function(x, K0, K1, K2, K3, K4) :
    return (K0-(k1/x)-K2*x+(K3*np.log(x))+(K4*np.log(x)))

In [ ]:
popt, pcov=curve_fit(guess_function, values["SoC"], values["Voc"], p0=(, , , , ), maxfev=5000)

In [ ]:
t= np.linspace(0, 1, 2000)
plt.plot(values["SoC"], values["Voc"], 'co', markersize=5)
plt.ylim(0, 1.9)
plt.xlim(0, 1.1)
plt.xlabel("$SoC(%)$", fontsize=13)
plt.ylabel("$V_{OC}$", fontsize=13)
plt.title("$V_{OC} vs SoC(%)$", fontsize=15)
plt.plot(t, guess_function(t, popt[0], popt[1], popt[2], popt[3], popt[4]),'r-' )
popt
plt.show()

#### We get the curve fitting parameters as follows

$$\theta = \begin{bmatrix} K_0 \\ K_1 \\ K_2 \\ K_3 \\ K_4 \end{bmatrix}=\begin{bmatrix}  K_0 \\ K_1 \\ K_2 \\ K_3 \\ K_4 \end{bmatrix}$$
##### Covariance for the fitting 

In [ ]:
pcov
plt.imshow(np.log(np.abs(pcov)))
plt.colorbar()
plt.title('Covariance ColourBar')
plt.show()

## <center> 🔋Notebook Concludes 📖